In [1]:
!apt update
!apt install chromium-chromedriver
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [973 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,994 kB]
Get:13 http://security.ubuntu.

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=options)

In [11]:
def scrape_page(url):
  browser.get(url)
  title = browser.find_elements(By.CSS_SELECTOR, ".product-title")[1].text or browser.find_elements(By.CSS_SELECTOR, ".product-title")[0].text
  full_price = '0.0'
  discount_price = '0.0'
  if browser.find_elements(By.CSS_SELECTOR, ".discount-percent"):
    full_price = browser.find_elements(By.CSS_SELECTOR, ".old-price")[1].text or browser.find_elements(By.CSS_SELECTOR, ".old-price")[0].text
    discount_price = browser.find_elements(By.CSS_SELECTOR, ".price")[1].text or browser.find_elements(By.CSS_SELECTOR, ".price")[0].text
  else:
    full_price = browser.find_elements(By.CSS_SELECTOR, ".price")[1].text or browser.find_elements(By.CSS_SELECTOR, ".price")[0].text

  full_price = full_price.replace(',', '.').replace(' EUR', '')
  discount_price = discount_price.replace(',', '.').replace(' EUR', '')

  color_options = []
  for i in range(0, len(browser.find_elements(By.CSS_SELECTOR, ".color-option:not(.disabled)"))):
    thubnail = browser.find_elements(By.CSS_SELECTOR, ".color-option:not(.disabled)>.color-box>div")[i].get_attribute('data-bg')
    color = browser.find_elements(By.CSS_SELECTOR, ".color-option:not(.disabled)")[i].text
    if i != 0:
      option = browser.find_elements(By.CSS_SELECTOR, ".color-option:not(.disabled)")[i]
      browser.get(option.get_attribute('href'))
    code = browser.find_elements(By.CSS_SELECTOR, ".product-code")[0].text or browser.find_elements(By.CSS_SELECTOR, ".product-code")[1].text
    code = code.split('\n')[1]
    model_size = browser.find_elements(By.CSS_SELECTOR, ".seo-detail")
    model_size = model_size[0].text if model_size and model_size[0].text.startswith('Chest') else '/'
    color_options.append((code, color, thubnail, model_size, [image.get_attribute('data-src') for image in browser.find_elements(By.CSS_SELECTOR, ".product-images-desktop>div>img")]))

  description = browser.find_elements(By.CSS_SELECTOR, ".panel-body>div>div>ul>li")
  description = description[0].text.replace(',', ' ').replace('\n', '; ') or '/'

  description_details = [a.text for a in browser.find_elements(By.CSS_SELECTOR, ".seo-detail")]
  description_details = description_details[1:] if description_details and description_details[0].startswith('Chest') else description_details or '/'

  washing_instructions = [element.get_attribute('innerText') for element in browser.find_elements(By.CSS_SELECTOR, ".panel-body>table>tbody>tr>td")][0::2] or '/'

  if browser.find_elements(By.CSS_SELECTOR, ".breadcrumb>li")[1].text == 'SALE %':
    clothing_category = browser.find_elements(By.CSS_SELECTOR, ".breadcrumb>li")[3].text.upper().replace("-", "_").replace(" ", "_")
    person_category = browser.find_elements(By.CSS_SELECTOR, ".breadcrumb>li")[2].text.upper()
  else:
    clothing_category = browser.find_elements(By.CSS_SELECTOR, ".breadcrumb>li")[2].text.upper().replace("-", "_").replace(" ", "_")
    person_category = browser.find_elements(By.CSS_SELECTOR, ".breadcrumb>li")[1].text.upper()

  return (title, full_price, discount_price, color_options, description, description_details, washing_instructions, clothing_category, person_category)

In [12]:
file = open('urls.txt','r')
content = file.read()
urls_to_scrape = content.split('\n')

data = []
for url in urls_to_scrape:
  data.append(scrape_page(url))

In [13]:
products = []
product_color_options = []
color_option_images = []

for product in data:
    title, full_price, discount_price, color_options, description, description_details, washing_instructions, clothing_category, person_category = product

    # Product data
    product_id = len(products) + 1
    products.append((product_id, title, full_price, discount_price, description, '; '.join(description_details), '; '.join(washing_instructions), clothing_category, person_category))

    # ColorOption data
    for option in color_options:
        code, color, thumbnail, model_size, images = option
        color_option_id = len(product_color_options) + 1
        product_color_options.append((color_option_id, product_id, code, color, thumbnail, model_size))

        # ColorOptionImages data
        for image in images:
            image_id = len(color_option_images) + 1
            color_option_images.append((image_id, color_option_id, image))

df_products = pd.DataFrame(products, columns=['Product ID', 'Title', 'Full Price', 'Discount Price', 'Description', 'Description Details', 'Washing Instructions', 'Clothing Category', 'Person Category'])
df_product_color_options = pd.DataFrame(product_color_options, columns=['Color Option ID', 'Product ID', 'Code', 'Color', 'Thumbnail', 'Model Size'])
df_color_option_images = pd.DataFrame(color_option_images, columns=['Image ID', 'Color Option ID', 'Image URL'])

In [14]:
df_products.to_csv("products.csv", index=False)
df_product_color_options.to_csv("product_color_options.csv", index=False)
df_color_option_images.to_csv("color_option_images.csv", index=False)